### Recipe Recommender Model
A more complicated model that I'll be using for my Recipe Book application. Its job is to look at the different recipes that the user has created or favourited in the past, and using that, recommend them other recipes when they browse for recipes that other people have made.

Input: User-made recipes, including names, description, ingredients, and steps. Along with this, recipes that other people have made.

Output: A score given to each recipe for how relevant they seem to the user.

Success Rate Goal: 90% accuracy/confidence.

In [17]:
# Imports
import torch
import torch.nn as nn

import numpy as np
import random
import time
import copy

import matplotlib.pyplot as plt

In [18]:
# Setting torch to use GPU acceleration if possible.
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")

torch.set_default_device(device)
print(f"Using device: {torch.get_default_device()}")

Using device: cpu


In [19]:
# ====================== DATA COLLECTION ======================

In [47]:
# Read all of the data.
lines = []

with open("data.txt", "r") as file:
    for line in file.readlines():
        lines.append(line.strip())

In [48]:
cIndex = 0
while cIndex < len(lines):
    if len(lines[cIndex]) > 100:
        print(cIndex)

    cIndex += 4

In [49]:
# Parse all of the data into recipes.
recipes = []

index = 0

while index < len(lines) - 3:
    recipes.append({
        "name": lines[index],
        "description": lines[index + 1],
        "ingredients": lines[index + 2],
        "steps": lines[index + 3]
    })

    index += 4

In [50]:
# Create the training and testing samples.
SAMPLES = 1000
TRAIN_RATIO = 0.9

RECIPE_MEDIAN = 5
RECIPE_VARIANCE = 3

RECIPE_TEST_MEDIAN = 10
RECIPE_TEST_VARIANCE = 5

dataSamples = []

for i in range(SAMPLES):
    cDataSample = [[], []]
    
    recipeCount = random.choice(range(RECIPE_MEDIAN - RECIPE_VARIANCE, RECIPE_MEDIAN + RECIPE_VARIANCE + 1))
    testCount = random.choice(range(RECIPE_TEST_MEDIAN - RECIPE_TEST_VARIANCE, RECIPE_TEST_MEDIAN + RECIPE_TEST_VARIANCE + 1))

    for j in range(recipeCount + testCount):
        chosenRecipe = copy.deepcopy(random.choice(recipes))
        chosenRecipe["favourites"] = random.choice(range(0, 1001))
        cDataSample[0 if (j >= recipeCount) else 1].append(chosenRecipe)

    dataSamples.append(cDataSample)

xTrain = dataSamples[:int(SAMPLES * TRAIN_RATIO)]
xTest = dataSamples[int(SAMPLES * TRAIN_RATIO):]

In [52]:
cDataSample

[[{'name': 'Baked aloo gobi with flatbreads',
   'description': 'This vegetarian whole-baked cauliflower and potato curry can also be made suitable for vegans.<br><br>Each serving provides 791 kcal, 20.5g protein, 82g carbohydrates (of which 12g sugars), 40.5g fat (of which 29g saturates), 9g fibre and 0.7g salt.',
   'ingredients': '1 large cauliflower, left whole|4 tbsp coconut oil|2.5cm/1in piece fresh root ginger, peeled and grated|4 green chillies, finely chopped and seeds removed|4 garlic cloves, crushed|1 tbsp black mustard seeds|2 tsp ground turmeric|400ml tin coconut milk|1 unwaxed lemon, halved|600g/1lb 5oz potatoes, scrubbed, skin left on and cut into 2cm/.75in pieces|salt and freshly ground black pepper|50g/1.75oz Greek-style yoghurt (or a thick dairy-free alternative), to serve|25g/1oz blanched almonds, to serve|coriander leaves, to serve|200g/7oz plain flour, plus extra for dusting |1 tsp baking powder |200g/7oz Greek-style yoghurt or 150ml/5fl oz warm water',
   'steps':